<a href="https://colab.research.google.com/github/rodericklivelo/Roderick-BUS-316-Final-Project/blob/main/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PreProcessing

In [16]:
pip install nba_api

In [53]:
import nba_api
import pandas as pd
import numpy as np
from nba_api.stats.endpoints import LeagueDashPlayerStats
from datetime import date
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error
from math import sqrt

In [27]:
mvp = pd.read_csv('/content/wow.csv')
mvp.head()

,Rank,PLAYER_NAME,MVP_SHARE,MVP,Season
0,1,Joel Embiid,0.915,1,2022-23
1,2,Nikola Jokic,0.674,0,2022-23
2,3,Giannis Antetokounmpo,0.606,0,2022-23
3,4,Jayson Tatum,0.280,0,2022-23
4,5,Shai Gilgeous-Alexander,0.046,0,2022-23


In [28]:
# Function to get season from a given year
def get_season(year):
    if date(year, 1, 1) < date(year, 7, 1):
        return str(year) + '-' + str(year+1)[-2:]
    else:
        return str(year-1) + '-' + str(year)[-2:]

In [29]:
# List to store dataframes
dfs = []

In [30]:
# Get data for each of the last 5 seasons
for year in range(date.today().year - 6, date.today().year-1):
    season = get_season(year)
    data = LeagueDashPlayerStats(season=season, season_type_all_star="Regular Season", per_mode_detailed="PerGame").get_data_frames()[0]
    data['Season'] = season
    dfs.append(data)

In [31]:
# Concatenate all dataframes
result_df = pd.concat(dfs)

In [32]:
# Reset index
result_df.reset_index(drop=True, inplace=True)

In [33]:
# Display the result dataframe
print(result_df)

      PLAYER_ID      PLAYER_NAME   NICKNAME     TEAM_ID TEAM_ABBREVIATION  \
0        203932     Aaron Gordon      Aaron  1610612753               ORL   
1       1628988    Aaron Holiday      Aaron  1610612754               IND   
2       1627846      Abdel Nader      Abdel  1610612760               OKC   
3        201143       Al Horford         Al  1610612738               BOS   
4        202329  Al-Farouq Aminu  Al-Farouq  1610612757               POR   
...         ...              ...        ...         ...               ...   
2738    1628380     Zach Collins       Zach  1610612759               SAS   
2739     203897      Zach LaVine       Zach  1610612741               CHI   
2740    1630192       Zeke Nnaji       Zeke  1610612743               DEN   
2741    1630533  Ziaire Williams     Ziaire  1610612763               MEM   
2742    1629627  Zion Williamson       Zion  1610612740               NOP   

       AGE  GP   W   L  W_PCT  ...  BLKA_RANK  PF_RANK  PFD_RANK  PTS_RANK 

In [83]:
necessary = result_df.query('MIN > 20' and 'PTS > 20')

In [72]:
merged_df = pd.merge(necessary, mvp, how='left', left_on=['PLAYER_NAME', 'Season'], right_on=['PLAYER_NAME', 'Season'])
merged_df['MVP_SHARE'].fillna(0, inplace=True)
merged_df['MVP'].fillna(0, inplace=True)
merged_df['MVP'] = merged_df['MVP'].astype(str)

In [ ]:
print(merged_df.sort_values(by='MVP_SHARE', ascending=False))

In [145]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204 entries, 0 to 203
Data columns (total 70 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PLAYER_ID              204 non-null    int64  
 1   PLAYER_NAME            204 non-null    object 
 2   NICKNAME               204 non-null    object 
 3   TEAM_ID                204 non-null    int64  
 4   TEAM_ABBREVIATION      204 non-null    object 
 5   AGE                    204 non-null    float64
 6   GP                     204 non-null    int64  
 7   W                      204 non-null    int64  
 8   L                      204 non-null    int64  
 9   W_PCT                  204 non-null    float64
 10  MIN                    204 non-null    float64
 11  FGM                    204 non-null    float64
 12  FGA                    204 non-null    float64
 13  FG_PCT                 204 non-null    float64
 14  FG3M                   204 non-null    float64
 15  FG3A  

# Trial 1

In [355]:
X = merged_df[['REB', 'AST', 'PTS']]
y = merged_df.MVP_SHARE

In [356]:
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.25, random_state = 42)

In [357]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [358]:
y_preds = model.predict(X_train)
mse = mean_squared_error(y_train, y_preds)
rmse = sqrt(mse)
print(rmse)

0.15436710753445068


In [359]:
X = merged_df[['REB', 'AST', 'PTS']]
y = merged_df.MVP

In [360]:
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.25, random_state = 42)

In [361]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [362]:
y_pred = model.predict(X_test)

In [363]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Confusion Matrix:
 [[49  0]
 [ 1  1]]


# Trial 2

In [364]:
X = merged_df[['REB', 'AST', 'PTS', 'STL', 'BLK', 'FG_PCT', 'MIN', 'TOV']]
y = merged_df.MVP_SHARE

In [365]:
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.25, random_state = 42)

In [366]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [367]:
y_preds = model.predict(X_train)
mse = mean_squared_error(y_train, y_preds)
rmse = sqrt(mse)
print(rmse)

0.1478446596524369


In [368]:
X = merged_df[['REB', 'AST', 'PTS', 'STL', 'BLK', 'FG_PCT', 'MIN', 'TOV']]
y = merged_df.MVP

In [369]:
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.25, random_state = 42)

In [370]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [371]:
y_pred = model.predict(X_test)

In [372]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Confusion Matrix:
 [[49  0]
 [ 1  1]]


# Trial 3

In [373]:
X = merged_df[['REB', 'AST', 'PTS', 'STL', 'BLK', 'FG_PCT', 'MIN', 'TOV', 'W_PCT']]
y = merged_df.MVP_SHARE

In [374]:
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.25, random_state = 42)

In [375]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [376]:
y_preds = model.predict(X_train)
mse = mean_squared_error(y_train, y_preds)
rmse = sqrt(mse)
print(rmse)

0.1443738977226969


In [377]:
X = merged_df[['REB', 'AST', 'PTS', 'STL', 'BLK', 'FG_PCT', 'MIN', 'TOV', "W_PCT"]]
y = merged_df.MVP

In [378]:
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.25, random_state = 42)

In [379]:
model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [380]:
y_pred = model.predict(X_test)

In [381]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Confusion Matrix:
 [[49  0]
 [ 1  1]]


# Trial 4

In [382]:
X = merged_df[['W_PCT_RANK', 'FG_PCT_RANK', 'AST_RANK', 'REB_RANK', 'TOV_RANK', 'BLK_RANK', 'STL_RANK', 'PTS_RANK']]
y = merged_df.MVP_SHARE

In [383]:
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.25, random_state = 42)

In [384]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [385]:
y_preds = model.predict(X_train)
mse = mean_squared_error(y_train, y_preds)
rmse = sqrt(mse)
print(rmse)

0.16229970509696337


In [386]:
X = merged_df[['W_PCT_RANK', 'FG_PCT_RANK', 'AST_RANK', 'REB_RANK', 'TOV_RANK', 'BLK_RANK', 'STL_RANK', 'PTS_RANK']]
y = merged_df.MVP

In [387]:
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.25, random_state = 42)

In [388]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [389]:
y_pred = model.predict(X_test)

In [390]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Confusion Matrix:
 [[49  0]
 [ 2  0]]


# Trial 5

In [146]:
X = merged_df[['REB', 'AST', 'PTS', 'STL', 'BLK', 'FG_PCT', 'MIN', 'TOV', 'W_PCT_RANK', 'FG_PCT_RANK', 'AST_RANK', 'REB_RANK', 'TOV_RANK', 'BLK_RANK', 'STL_RANK', 'PTS_RANK']]
y = merged_df.MVP_SHARE

In [147]:
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.25, random_state = 42)

In [148]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [149]:
y_preds = model.predict(X_train)
mse = mean_squared_error(y_train, y_preds)
rmse = sqrt(mse)
print(rmse)

0.13437153020303383


In [150]:
X = merged_df[['REB', 'AST', 'PTS', 'STL', 'BLK', 'FG_PCT', 'MIN', 'TOV', 'W_PCT_RANK', 'FG_PCT_RANK', 'AST_RANK', 'REB_RANK', 'TOV_RANK', 'BLK_RANK', 'STL_RANK', 'PTS_RANK']]
y = merged_df.MVP

In [151]:
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.25, random_state = 42)

In [152]:
model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [153]:
y_pred = model.predict(X_test)

In [154]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Confusion Matrix:
 [[49  0]
 [ 2  0]]
